In [ ]:
# 📌 Instalación de librerías necesarias
!pip install pandas numpy scikit-learn nltk

In [3]:
# 📌 Importar Librerías Necesarias
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import json

# 📌 Cargar los Datos desde archivos CSV
import os
ruta_archivos = 'CSV/Por Marcas'  # Ruta donde están almacenados los archivos CSV

# Obtener la lista de archivos en la carpeta especificada
archivos = [archivo for archivo in os.listdir(ruta_archivos) if archivo.endswith('.csv')]

# Leer y unir todos los CSV en un solo DataFrame
df_lista = []
for archivo in archivos:
    df = pd.read_csv(os.path.join(ruta_archivos, archivo))  # Leer cada archivo CSV
    df_lista.append(df)

df_celulares = pd.concat(df_lista, ignore_index=True)  # Unir todos los DataFrames en uno solo

# 📌 Eliminar Columnas de Precios No Deseadas
columnas_a_eliminar = ['Launched Price (Pakistan)', 'Launched Price (India)', 'Launched Price (China)', 'Launched Price (Dubai)']
df_celulares.drop(columns=[col for col in columnas_a_eliminar if col in df_celulares.columns], inplace=True)  # Eliminar solo si existen

# 📌 Renombrar Columnas para un mejor entendimiento
df_celulares.columns = ['Company Name', 'Model Name', 'Mobile Weight (g)', 'RAM (GB)', 'Front Camera (MP)',
                        'Back Camera (MP)', 'Processor', 'Battery Capacity (mAh)', 'Screen Size (inches)', 'Launched Price (USA)', 'Launched Year']

# 📌 Limpieza y Conversión de Datos

df_celulares.dropna(inplace=True)  # Eliminar filas con valores nulos

# Convertir nombres de modelos y procesadores a minúsculas para estandarizar
df_celulares['Model Name'] = df_celulares['Model Name'].str.lower()
df_celulares['Processor'] = df_celulares['Processor'].str.lower()

# Convertir pesos de móviles a valores numéricos
df_celulares['Mobile Weight (g)'] = df_celulares['Mobile Weight (g)'].str.replace('g', '', regex=True).astype(float)

# 📌 Expandir Variantes de RAM en diferentes filas

def expandir_variantes_ram(df):
    """
    Función que separa valores de RAM expresados en múltiplas variantes (ej. "4GB/6GB/8GB")
    y los convierte en filas individuales.
    """
    nuevas_filas = []
    for _, fila in df.iterrows():
        valores_ram = [float(x) for x in str(fila['RAM (GB)']).replace('GB', '').split('/') if x.strip().isdigit()]
        for ram in valores_ram:
            nueva_fila = fila.copy()
            nueva_fila['RAM (GB)'] = ram
            nuevas_filas.append(nueva_fila)
    return pd.DataFrame(nuevas_filas)

df_celulares = expandir_variantes_ram(df_celulares)

# 📌 Extraer megapíxeles de las cámaras

def extraer_megapixeles(valor):
    """
    Función que extrae el valor numérico de megapíxeles de una cadena de texto (ej. "12MP/16MP").
    Devuelve el primer valor encontrado.
    """
    valores = [float(x) for x in valor.replace('MP', '').split('/') if x.strip().replace('.', '').isdigit()]
    return valores[0] if valores else np.nan

df_celulares['Front Camera (MP)'] = df_celulares['Front Camera (MP)'].astype(str).apply(extraer_megapixeles)
df_celulares['Back Camera (MP)'] = df_celulares['Back Camera (MP)'].astype(str).apply(extraer_megapixeles)

# 📌 Limpieza y conversión de la capacidad de batería
df_celulares['Battery Capacity (mAh)'] = (
    df_celulares['Battery Capacity (mAh)']
    .str.replace(',', '', regex=True)  # Eliminar comas para valores numéricos en miles
    .str.replace('mAh', '', regex=True)  # Eliminar la unidad de medida
    .astype(float)  # Convertir a número flotante
)

# 📌 Extraer el tamaño de pantalla
def extraer_tamano_pantalla(valor):
    """
    Función que extrae el tamaño numérico de la pantalla desde una cadena de texto,
    ignorando posibles anotaciones adicionales como "(unfolded)".
    """
    import re
    match = re.search(r'\d+(\.\d+)?', valor)
    return float(match.group()) if match else np.nan

df_celulares['Screen Size (inches)'] = df_celulares['Screen Size (inches)'].astype(str).apply(extraer_tamano_pantalla)

# 📌 Limpieza y conversión del precio de lanzamiento
df_celulares['Launched Price (USA)'] = (
    df_celulares['Launched Price (USA)']
    .str.replace(',', '', regex=True)  # Eliminar comas para valores en miles
    .str.replace('USD ', '', regex=True)  # Eliminar prefijo "USD"
    .astype(float)  # Convertir a número flotante
)

# 📌 Mostrar los datos después de la limpieza
print("Datos después de la limpieza y conversión:")
display(df_celulares.head())

# 📌 Guardar el dataset limpio en formato JSON
ruta_salida = 'celulares_limpios.json'  # Nombre del archivo JSON
df_celulares.to_json(ruta_salida, orient='records', indent=4)

print(f"📁 Dataset guardado en {ruta_salida}")


Datos después de la limpieza y conversión:


,Company Name,Model Name,Mobile Weight (g),RAM (GB),Front Camera (MP),Back Camera (MP),Processor,Battery Capacity (mAh),Screen Size (inches),Launched Price (USA),Launched Year
0,Apple,iphone 16 128gb,174.0,6.0,12.0,48.0,a17 bionic,3600.0,6.1,799.0,2024
1,Apple,iphone 16 256gb,174.0,6.0,12.0,48.0,a17 bionic,3600.0,6.1,849.0,2024
2,Apple,iphone 16 512gb,174.0,6.0,12.0,48.0,a17 bionic,3600.0,6.1,899.0,2024
3,Apple,iphone 16 plus 128gb,203.0,6.0,12.0,48.0,a17 bionic,4200.0,6.7,899.0,2024
4,Apple,iphone 16 plus 256gb,203.0,6.0,12.0,48.0,a17 bionic,4200.0,6.7,949.0,2024


📁 Dataset guardado en celulares_limpios.json
